In [47]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
import math
from tqdm import tqdm
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
warnings.filterwarnings('ignore')

import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:] #So this dview (direct viewing) becomes the parallelizing program-direct execution in cloud

[0, 1, 2, 3]


In [48]:
import re

In [49]:
einlist = pd.read_csv("/Users/Wolfalev/Documents/Grad/UT-AUSTIN GRAD/Coursework/CUSS/JiRepo/npo_classifier/dataset/eNGO_TX_ein_name_list.csv")

#for ein in einlist['EIN']:
 #   print(ein)

In [50]:
#xml_runner = XMLRunner()
#parsed_filing = xml_runner.run_sked(2012134249347, 'IRS990ScheduleJ')

### Need to get object ID

### Prepare UK Data.

```Python
# Compress icnpo training data.
df_icnpo_classifier_training_data=pd.read_csv('../dataset/icnpo_classifier_training_data.csv', sep=',')
df_icnpo_classifier_training_data.to_pickle('../dataset/icnpo_classifier_training_data.pkl.gzip', compression='gzip')
```

### Prepare US Training Data.
Useful links:
- IRS 990 forms on AWS: https://registry.opendata.aws/irs990/
- NCCS Data Archive (data dict, NTEE codes, etc.): https://nccs-data.urban.org/index.php

### TODO:
- [+] Add year of formation. - Do not use information from 990 forms: only Form 990 has "Year of Formation". Use BMF 199508 to dummy-code the organizations.
- [ ] Add program descriptions in Part VIII of Form 990.

In [ ]:
df_index=pd.DataFrame()
for year in range(2013, 2019):
        try:
            print(str(year), 'started', end='\t')
            df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_'+str(year)+'.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 )
            df_index_temp['YEAR']=year
            df_index=pd.concat([df_index, df_index_temp], ignore_index=True)
            print('got')
        except:
            print('passed')

2013 started	got
2014 started	

b'Skipping line 39569: expected 9 fields, saw 10\n'


got
2015 started	got
2016 started	got
2017 started	got
2018 started	

In [ ]:
df_index.groupby(['YEAR','RETURN_TYPE']).count()

### Acquiring text data.

In [ ]:
df_index_matched = df_index[df_index['EIN'].isin(einlist['EIN'])]
                       
len(df_index_matched)                                               

In [ ]:
df_index_matched.sample(10)

In [ ]:
done_index_list=[]
dview.execute('import pandas as pd')
dview.execute('from time import sleep')
dview.execute('from irsx.xmlrunner import XMLRunner')
dview.execute('import os')
dview['xml_runner'] = XMLRunner()
dview['df_index'] = df_index_matched
dview['df_index_text_acq'] = pd.DataFrame()

In [ ]:
dview['done_index_list'] = list(set(done_index_list))
dview['counter']=0

In [ ]:
###### Define functions ######################################
def func_irs990_text(obj_id):
    IRS990_result=xml_runner.run_sked(obj_id, 'IRS990').result 

    ############## Check filing version.##############
    if IRS990_result!=None:
        IRS990_result=IRS990_result[0]
    else:
        return {'IRS990_p1_ActvtyOrMssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_MssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_DscS': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ##################################################

    ###### IRS990_p1_ActvtyOrMssnDsc #################
    IRS990_p1_ActvtyOrMssnDsc=IRS990_result['schedule_parts']['part_i']['ActvtyOrMssnDsc']
    ##################################################

    ##### IRS990_p3_MssnDsc, IRS990_p3_DscS ##########
    IRS990_p3=IRS990_result['schedule_parts']['part_iii']
    IRS990_p3_MssnDsc='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if key=='MssnDsc'])
    IRS990_p3_DscS='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if (key=='Dsc' or '_Dsc' in key)])
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990_p3.keys() and IRS990_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990_p1_ActvtyOrMssnDsc': IRS990_p1_ActvtyOrMssnDsc,
            'IRS990_p3_MssnDsc': IRS990_p3_MssnDsc,
            'IRS990_p3_DscS': IRS990_p3_DscS,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990ez_text(obj_id):
    IRS990EZ_result=xml_runner.run_sked(obj_id, 'IRS990EZ').result

    ############## Check filing version.##############
    if IRS990EZ_result!=None:
        IRS990EZ_result=IRS990EZ_result[0]
    else:
        return {'IRS990EZ_p3_PrmryExmptPrpsTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ###### IRS990EZ_p3_PrmryExmptPrpsTxt #############
    if 'ez_part_iii' in IRS990EZ_result['schedule_parts'].keys():
        IRS990EZ_p3=IRS990EZ_result['schedule_parts']['ez_part_iii']
        IRS990EZ_p3_PrmryExmptPrpsTxt='##'.join([IRS990EZ_p3[key] for key in IRS990EZ_p3.keys() if key=='PrmryExmptPrpsTxt'])
    else:
        IRS990EZ_p3_PrmryExmptPrpsTxt=''
    ##################################################

    ####### IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt ######
    if 'EZPrgrmSrvcAccmplshmnt' in IRS990EZ_result['groups'].keys():
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt='##'.join([s['DscrptnPrgrmSrvcAccmTxt'] for s in IRS990EZ_result['groups']['EZPrgrmSrvcAccmplshmnt']])
    else:
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt=''
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990EZ_p3.keys() and IRS990EZ_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990EZ_p3_PrmryExmptPrpsTxt': IRS990EZ_p3_PrmryExmptPrpsTxt,
            'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990pf_text(obj_id):
    IRS990PF_result=xml_runner.run_sked(obj_id, 'IRS990PF').result

    ############## Check filing version.##############
    if IRS990PF_result!=None:
        IRS990PF_result=IRS990PF_result[0]
    else:
        return {'IRS990PF_p9a_DscrptnTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990PF_p16b_RltnshpSttmntTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ##### IRS990PF_p9a_DscrptnTxt ##########################
    if 'pf_part_ixa' in IRS990PF_result['schedule_parts'].keys():
        IRS990PF_p9a=IRS990PF_result['schedule_parts']['pf_part_ixa']
        IRS990PF_p9a_DscrptnTxt='##'.join([IRS990PF_p9a[key] for key in IRS990PF_p9a.keys() if 'Txt' in key])
    else:
        IRS990PF_p9a_DscrptnTxt=''
    ########################################################

    ##### IRS990PF_p16b_RltnshpSttmntTxt ###################
    if 'PFRlnOfActyTAccmOfExmptPrps' in IRS990PF_result['groups']:
        IRS990PF_p16b_RltnshpSttmntTxt='##'.join([s['RltnshpSttmntTxt'] for s in IRS990PF_result['groups']['PFRlnOfActyTAccmOfExmptPrps']])
    else:
        IRS990PF_p16b_RltnshpSttmntTxt=''
    ########################################################

    return {'IRS990PF_p9a_DscrptnTxt':IRS990PF_p9a_DscrptnTxt,
            'IRS990PF_p16b_RltnshpSttmntTxt':IRS990PF_p16b_RltnshpSttmntTxt,
           }
###### Define functions ######################################

In [ ]:
###### Run main function ################################
df_index_text_acq = pd.DataFrame()
df_index = pd.DataFrame()
df_index = df_index_matched
done_index_list = []
for index in df_index_matched.index:
    try:
        if index not in done_index_list: #Index is position of filing in list of all filings in index file
            obj_id = df_index.loc[index, 'OBJECT_ID'] #if that position is not filled, pull the object id
            return_type=df_index.loc[index, 'RETURN_TYPE'] #if that position is not filled, pull the return type
            if return_type in ['990', '990O']: # (1) if return type  is 990, do the following
                text_dict=func_irs990_text(obj_id) #apply this function for 990s, using the object id from index to get the 990 in a text_dict
                df_index.loc[index, 'IRS990_p1_ActvtyOrMssnDsc']=text_dict['IRS990_p1_ActvtyOrMssnDsc'] #get mission and put it in index file
                df_index.loc[index, 'IRS990_p3_MssnDsc']=text_dict['IRS990_p3_MssnDsc'] # get mssndsc and put it here
                df_index.loc[index, 'IRS990_p3_DscS']=text_dict['IRS990_p3_DscS'] # get DsC and put it here
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']# get schedule 0 and put it here
            elif return_type in ['990EZ', '990EO']: #(2) same thing
                text_dict=func_irs990ez_text(obj_id)
                df_index.loc[index, 'IRS990EZ_p3_PrmryExmptPrpsTxt']=text_dict['IRS990EZ_p3_PrmryExmptPrpsTxt']
                df_index.loc[index, 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']=text_dict['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']
            elif return_type in ['990PF']: #(3) same thing
                text_dict=func_irs990pf_text(obj_id)
                df_index.loc[index, 'IRS990PF_p9a_DscrptnTxt']=text_dict['IRS990PF_p9a_DscrptnTxt']
                df_index.loc[index, 'IRS990PF_p16b_RltnshpSttmntTxt']=text_dict['IRS990PF_p16b_RltnshpSttmntTxt']
            df_index_text_acq=pd.concat([df_index_text_acq, df_index.loc[[index]]], ignore_index=True) #take the entries we just pulled, turn into df for that row, and concatenate them together into df_index_text_acq
            done_index_list+=[index] ## update that we've done this many indicies. 
    except:
        pass

In [ ]:
len(set(done_index_list)), len(df_index_text_acq)

In [ ]:
df_index_text_acq.sample(10)

In [ ]:
#df_index_text_acq_2018= df_index_text_acq[df_index_text_acq.YEAR == 2018]
#All Years

In [ ]:
df_index_text_acq.sample(10)
print(len(df_index_text_acq))

len(df_EIN_TXT), len(df_index_matched)

In [ ]:
df_bmf_1812=pd.read_csv('https://nccs-data.urban.org/data/bmf/2018/bmf.bm1812.csv')
df_bmf_1712=pd.read_csv('https://nccs-data.urban.org/data/bmf/2017/bmf.bm1712.csv')
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1312=pd.read_csv('https://nccs-data.urban.org/data/bmf/2013/bmf.bm1312.csv')

In [ ]:
df_bmf_1812['YEAR_NCCS'] = 2018
df_bmf_1712['YEAR_NCCS'] = 2017
df_bmf_1608['YEAR_NCCS'] = 2016
df_bmf_1512['YEAR_NCCS'] = 2015
df_bmf_1412['YEAR_NCCS'] = 2014
df_bmf_1312['YEAR_NCCS'] = 2013

#df_bmf_1812['Real_Year'] = df_bmf_1812['TAXPER'].astype(str).str[:4]
#df_bmf_1712['Real_Year'] = df_bmf_1712['TAXPER'].astype(str).str[:4]
#df_bmf_1608['Real_Year'] = df_bmf_1608['TAXPER'].astype(str).str[:4]
#df_bmf_1512['Real_Year'] = df_bmf_1512['TAXPER'].astype(str).str[:4]
#df_bmf_1412['Real_Year'] = df_bmf_1412['TAXPER'].astype(str).str[:4]
#df_bmf_1312['Real_Year'] = df_bmf_1312['TAXPER'].astype(str).str[:4]

#df_bmf_1812 = df_bmf_1812[df_bmf_1812['Real_Year'] == '2018']
#df_bmf_1712 = df_bmf_1712[df_bmf_1712['Real_Year'] == '2017']
#df_bmf_1608 = df_bmf_1608[df_bmf_1608['Real_Year'] == '2016']
#df_bmf_1512 = df_bmf_1512[df_bmf_1512['Real_Year'] == '2015']
#df_bmf_1412 = df_bmf_1412[df_bmf_1412['Real_Year'] == '2014']
#df_bmf_1312 = df_bmf_1312[df_bmf_1312['Real_Year'] == '2013']



In [ ]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

In [ ]:
#df_bmf_1812.drop_duplicates(subset=['TAXPER'],keep='last')
#df_bmf_1712.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1608.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1512.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1412.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1312.drop_duplicates(subset='EIN',keep='last')

In [ ]:
df_bmf_1812.columns

In [ ]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

In [ ]:
df_bmf_13_18=pd.concat([df_bmf_1312, df_bmf_1412, df_bmf_1512, df_bmf_1608, df_bmf_1712, df_bmf_1812], ignore_index=True)
df_bmf_13_18['Real_Year'] = df_bmf_13_18['TAXPER'].astype(str).str[:4]
df_bmf_13_18.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')
#remove any duplicated taxperiods within an ein, only taking  the last


In [ ]:
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.STATE == 'TX']
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.NTEE1 == 'C']

In [ ]:
df_bmf_13_18['EIN'] = pd.to_numeric(df_bmf_13_18['EIN'])
df_index_text_acq['EIN'] = pd.to_numeric(df_index_text_acq['EIN'])

In [ ]:
#df_bmf_13_18.drop_duplicates(subset=['EIN', 'CTAXPER'],keep='first')
#print(len(df_bmf_13_18))
df_index_text_acq['Real_Year'] = df_index_text_acq['TAX_PERIOD'].astype(str).str[:4]
df_index_text_acq.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')

print(len(df_index_text_acq))

In [ ]:
#df_bmf_13_18['YEAR_real'] = df_bmf_13_18['CTAXPER']
#df_index_text_acq['YEAR_real'] = df_index_text_acq['TAX_PERIOD']

In [ ]:
df_EIN_TXT=df_index_text_acq.merge(df_bmf_13_18, on=['EIN', 'Real_Year'] , how='outer')
df_EIN_TXT[df_EIN_TXT['Real_Year' > 2012]]
df_EIN_TXT.sample(3)

In [ ]:
df_EIN_TXT.columns

In [ ]:
print(len(df_EIN_TXT))

In [ ]:
df_EIN_TXT['EIN'].unique

In [ ]:
df_EIN_TXT['EIN'].isin(einlist['EIN']).value_counts()

In [ ]:
#df_EIN_TXT[['EIN', 'TAX_PERIOD']].drop_duplicates()
#len(df_EIN_TXT)

In [ ]:
df_EIN_TXT.to_csv('./texas_20132018_envorgs.csv', index=False)

In [ ]:
df_EIN_TXT_list = df_EIN_TXT[df_EIN_TXT['EIN'].isin(einlist['EIN'])]

In [ ]:
len(df_EIN_TXT_list)

In [ ]:
df_EIN_TXT_list.to_csv('./texas_20132018_envorgs_matched.csv', index=False)